In [2]:
!pip install transformers seqeval -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 12.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 4.1.1 requires pyarrow>=21.0.0, but you have pyarrow 19.0.1 which is incompatible.
gradio 5.38.1 requires pydantic<2.12,>=2.0, but you have pydantic 2.12.0a1 which is incompatible.


In [3]:
import json
import torch
import os
from torch.utils.data import Dataset, DataLoader
from seqeval.metrics import classification_report
from tqdm.notebook import tqdm
import numpy as np
from transformers import BertTokenizer, BertForTokenClassification
from torch.optim import AdamW
from transformers import (
    BertTokenizerFast,
    BertForTokenClassification
)

2025-10-30 23:14:07.601425: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761866047.769560      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761866047.819815      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
TRAIN_PATH = '/kaggle/input/mountai-train/mountains_dataset_large.json' 
TEST_PATH = '/kaggle/input/mountain-data/mountains_dataset.json'

In [5]:
MODEL_NAME = 'bert-base-multilingual-cased' 
MAX_LEN = 128
BATCH_SIZE = 16
LEARNING_RATE = 2e-5
EPOCHS = 4
OUTPUT_DIR = 'mountain_ner_model'
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
# --- Install required libraries ---
!pip install transformers seqeval -q

import json
import torch
import os
from torch.utils.data import Dataset, DataLoader
# Use BertTokenizerFast and import AdamW from torch.optim
from transformers import BertTokenizerFast, BertForTokenClassification
from torch.optim import AdamW
from seqeval.metrics import classification_report
from tqdm.notebook import tqdm
import numpy as np
import sys

# --- 0. Configuration and Mapping ---

# YOUR ACTUAL PATHS IN KAGGLE
TRAIN_PATH = '/kaggle/input/mountaintrain/ldataset_large_single_file_converted.json'
TEST_PATH = '/kaggle/input/testmountains/validation_dataset_array.json'

MODEL_NAME = 'bert-base-multilingual-cased'
MAX_LEN = 128
BATCH_SIZE = 16
LEARNING_RATE = 2e-5
EPOCHS = 3
OUTPUT_DIR = 'mountain_ner_model'
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

LABEL_LIST = ["O", "B-MOUNTAIN", "I-MOUNTAIN", "[CLS]", "[SEP]"]
label_map = {label: i for i, label in enumerate(LABEL_LIST)}
id_to_label = {i: label for label, i in label_map.items()}

print(f"Using device: {DEVICE}")

# --- 1. NERDataset Class ---

class NERDataset(Dataset):
    def __init__(self, data_path, tokenizer, label_map, max_len):
        self.tokenizer = tokenizer
        self.label_map = label_map
        self.max_len = max_len
        self.data = self._load_data(data_path)

        # Check to avoid ValueError: num_samples=0
        if not self.data:
            print(f"❌ Critical Error: Dataset at {data_path} is empty or not loaded.")

    def _load_data(self, data_path):
        if not os.path.exists(data_path):
            print(f"⚠️ File not found at path: {data_path}")
            return []
        try:
            with open(data_path, "r", encoding="utf-8") as f:
                data = json.load(f)
            print(f"✅ Successfully loaded {len(data)} records from {data_path}")
            return data
        except Exception as e:
            print(f"❌ Error reading JSON file {data_path}: {e}")
            return []

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sentence = self.data[idx]
        tokens = sentence["tokens"]
        labels = sentence["labels"]

        tokenized_input = self.tokenizer(
            tokens,
            is_split_into_words=True,
            padding="max_length",
            truncation=True,
            max_length=self.max_len,
            return_tensors="pt"
        )

        input_ids = tokenized_input["input_ids"].squeeze(0)
        attention_mask = tokenized_input["attention_mask"].squeeze(0)

        # Label alignment
        word_ids = tokenized_input.word_ids()
        label_ids = []
        previous_word_idx = None

        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(self.label_map.get(labels[word_idx], -100))
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx

        labels_tensor = torch.tensor(label_ids, dtype=torch.long)

        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": labels_tensor
        }

# --- 2. Evaluation Function ---

def evaluate_model(model, dataloader, device, id_to_label, phase="Evaluation"):
    model.eval()
    all_preds, all_labels = [], []

    for batch in tqdm(dataloader, desc=phase):
        with torch.no_grad():
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            preds = torch.argmax(outputs.logits, dim=-1).cpu().numpy()
            labels = labels.cpu().numpy()

            for i in range(labels.shape[0]):
                pred_tags, true_tags = [], []
                for j in range(labels.shape[1]):
                    if labels[i, j] != -100:
                        pred_tags.append(id_to_label[preds[i, j]])
                        true_tags.append(id_to_label[labels[i, j]])
                if true_tags:
                    all_preds.append(pred_tags)
                    all_labels.append(true_tags)

    print(f"\n--- {phase} ---")
    if all_labels:
        print(classification_report(all_labels, all_preds, digits=4))
    else:
        print("No data for evaluation.")
    return all_labels, all_preds

# --- 3. Training and Evaluation Function ---

def train_and_evaluate():
    # Initialize tokenizer and model
    tokenizer = BertTokenizerFast.from_pretrained(MODEL_NAME)
    
    model = BertForTokenClassification.from_pretrained(
        MODEL_NAME,
        num_labels=len(LABEL_LIST),
        id2label=id_to_label,
        label2id=label_map
    ).to(DEVICE)

    # Freeze first 8 layers
    FREEZE_LAYERS = 8
    for name, param in model.bert.named_parameters():
        if "encoder.layer." in name:
            layer_num = int(name.split("encoder.layer.")[1].split(".")[0])
            if layer_num < FREEZE_LAYERS:
                param.requires_grad = False
    print(f"Frozen first {FREEZE_LAYERS} BERT layers.")

    # --- Data ---
    train_dataset = NERDataset(TRAIN_PATH, tokenizer, label_map, MAX_LEN)
    test_dataset = NERDataset(TEST_PATH, tokenizer, label_map, MAX_LEN)
    
    # Check for empty dataset before DataLoader creation
    if not train_dataset or not test_dataset:
        print("Training stopped: one of the datasets is empty.")
        return None, None
        
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

    optimizer = AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=LEARNING_RATE)

    for epoch in range(EPOCHS):
        model.train()
        total_loss = 0
        for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}"):
            input_ids = batch["input_ids"].to(DEVICE)
            attention_mask = batch["attention_mask"].to(DEVICE)
            labels = batch["labels"].to(DEVICE)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss

            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            total_loss += loss.item()

        avg_loss = total_loss / len(train_loader)
        print(f"Epoch {epoch+1} | Average Loss = {avg_loss:.4f}")

    # --- Save model in Kaggle ---
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    model.save_pretrained(OUTPUT_DIR)
    tokenizer.save_pretrained(OUTPUT_DIR)
    print(f"✅ Model and tokenizer saved to: {OUTPUT_DIR}")

    evaluate_model(model, test_loader, DEVICE, id_to_label, "Final Evaluation")
    return model, tokenizer

# --- 4. Inference Function ---

def predict_mountain_names(text: str, model, tokenizer):
    model.eval()
    
    # Tokenize with is_split_into_words=True
    tokenized = tokenizer(text.split(), is_split_into_words=True, return_tensors="pt").to(DEVICE)
    
    with torch.no_grad():
        outputs = model(**tokenized)
        
    preds = torch.argmax(outputs.logits, dim=-1).squeeze(0).cpu().numpy()
    tokens = tokenizer.convert_ids_to_tokens(tokenized["input_ids"].squeeze(0).cpu().numpy())

    current_mountain_tokens = []
    results = []
    
    # Collect tokens
    for token, pred_id in zip(tokens, preds):
        label = id_to_label.get(pred_id, "O")
        
        # Skip special tokens [CLS] and [SEP]
        if token in tokenizer.all_special_tokens:
             continue
        
        if label.startswith("B-MOUNTAIN"):
            # Start of a new mountain entity
            if current_mountain_tokens: 
                results.append(" ".join(current_mountain_tokens))
            current_mountain_tokens = [token]
        elif label.startswith("I-MOUNTAIN"):
            # Continue current mountain entity
            current_mountain_tokens.append(token)
        else:  # O
            if current_mountain_tokens:
                results.append(" ".join(current_mountain_tokens))
                current_mountain_tokens = []
                
    # Add the last entity if any
    if current_mountain_tokens:
        results.append(" ".join(current_mountain_tokens))
        
    # Clean tokens and return unique mountain names
    cleaned_results = [
        r.replace(" ##", "")
        for r in results if r.strip()
    ]
    
    return list(set(cleaned_results))

# --- 5. Training and Demonstration ---

print("--- Starting training and saving the model ---")
trained_model, trained_tokenizer = train_and_evaluate()

if trained_model is None:
    print("\n⚠️ Unable to continue: training stopped due to empty data.")
else:
    print("\n--- Inference demonstration ---")
    test_sentences = [
        "I climbed Mount Everest last year with a group of friends.",
        "She dreams of hiking Mount Fuji during the cherry blossom season.",
        "The snow on Mount Kilimanjaro never melts completely.",
        "We took amazing photos near Mount Elbrus in Russia.",
        "Mount Denali is the highest mountain in North America.",
        "They plan to explore both Mount Etna and Mount Vesuvius this summer.",
        "The Andes and the Himalayas are two of the most famous mountain ranges.",
        "We traveled through the Alps, visiting both Mont Blanc and the Matterhorn.",
        "Mount Cook in New Zealand is popular among professional climbers.",
        "Table Mountain in South Africa offers an incredible view of Cape Town."
    ]

    for i, text in enumerate(test_sentences, 1):
        print(f"\nText {i}: {text}")
        if trained_model and trained_tokenizer:
            print(f"🏔️ Mountains: {predict_mountain_names(text, trained_model, trained_tokenizer)}")


Using device: cuda
--- Starting training and saving the model ---


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

...

Epoch 2 | Average Loss = 0.0018


Epoch 3/3:   0%|          | 0/188 [00:00<?, ?it/s]

Epoch 3 | Average Loss = 0.0010
✅ Model and tokenizer saved to: mountain_ner_model


Final Evaluation:   0%|          | 0/13 [00:00<?, ?it/s]


--- Final Evaluation ---
              precision    recall  f1-score   support

    MOUNTAIN     1.0000    0.9434    0.9709       106

   micro avg     1.0000    0.9434    0.9709       106
   macro avg     1.0000    0.9434    0.9709       106
weighted avg     1.0000    0.9434    0.9709       106


--- Inference demonstration ---

Text 1: I climbed Mount Everest last year with a group of friends.
🏔️ Mountains: ['Mount Everest']

Text 2: She dreams of hiking Mount Fuji during the cherry blossom season.
🏔️ Mountains: ['bl', 'Mount Fuji']

Text 3: The snow on Mount Kilimanjaro never melts completely.
🏔️ Mountains: ['Mount Kilimanjaro']

Text 4: We took amazing photos near Mount Elbrus in Russia.
🏔️ Mountains: ['Mount Elbrus']

Text 5: Mount Denali is the highest mountain in North America.
🏔️ Mountains: ['Mount Denali']

Text 6: They plan to explore both Mount Etna and Mount Vesuvius this summer.
🏔️ Mountains: ['Mount Etna', 'Mount Vesuvius']

Text 7: The Andes and the Himalayas are two of